In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from bs4 import BeautifulSoup

import re
import sys
import urllib.request

In [2]:
options = FirefoxOptions()
options.add_argument("--headless")

browser = webdriver.Firefox(executable_path="C:\\Users\\marco\\geckodriver.exe", options=options)

C:\Users\marco\AppData\Local\Temp\ipykernel_22200\3901147259.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path="C:\\Users\\marco\\geckodriver.exe", options=options)


In [66]:
def clean_url(link):
    # extract arxiv id from url
    arxiv_id = link.split('/')[-1]
    # remove .pdf from arxiv id if it exists
    arxiv_id = arxiv_id.replace('.pdf', '')
    # construct new ar5iv url 
    url = 'https://ar5iv.labs.arxiv.org/html/' + arxiv_id
    return url

def get_links(html):
    links = []
    # find all links that contain 'arxiv' in the url and end in .pdf or numbers
    for match in re.finditer(r'href=[\'"]?([^\'" >]+)', html):
        url = match.group(1)
        if 'arxiv' in url and ('.pdf' in url or url[-3].isdigit()):
            links.append(clean_url(url))
    return links


base_url = [
    'https://interactiveaudiolab.github.io/teaching/generative_deep_models.html', 
    'https://github.com/dair-ai/ML-Papers-of-the-Week'
]
links = []
# get list of urls ending in .pdf and containing 'arxiv' in the url
for url in base_url:
    browser.get(url)
    html = browser.page_source
    links += get_links(html)


print(f'sample: {links[0:5]}')
print(f'total: {len(links)}')

sample: ['https://ar5iv.labs.arxiv.org/html/2305.10973v1', 'https://ar5iv.labs.arxiv.org/html/2305.11169', 'https://ar5iv.labs.arxiv.org/html/2305.09617', 'https://ar5iv.labs.arxiv.org/html/2305.07185', 'https://ar5iv.labs.arxiv.org/html/2305.09645']
total: 182


In [2]:
browser = webdriver.Firefox(executable_path="C:\\Users\\marco\\geckodriver.exe")
url = "https://ar5iv.labs.arxiv.org/html/2112.10741"
browser.get(url)


C:\Users\marco\AppData\Local\Temp\ipykernel_47740\2675970555.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path="C:\\Users\\marco\\geckodriver.exe")


In [3]:
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Get the title
title = soup.find('h1', class_='ltx_title ltx_title_document').text.strip()
print(title)

GLIDE: Towards Photorealistic Image Generation and Editing with Text-Guided Diffusion Models


In [5]:
# Get the authors
authors = soup.find('div', class_='ltx_authors').find_all('span', class_='ltx_personname')
authors = [author.text.strip() for author in authors]
print(authors)

['Alex Nichol', 'Prafulla Dhariwal', 'Aditya Ramesh', 'Pranav Shyam', 'Pamela Mishkin', 'Bob McGrew', 'Ilya Sutskever', 'Mark Chen']


In [8]:
# Get the abstract
abstract = soup.find('div', class_='ltx_abstract').find('p').text.strip()
print(abstract[0:150])

Diffusion models have recently been shown to generate high-quality synthetic images, especially when paired with a guidance technique to trade off div


In [46]:
# Get each section and store it in a dictionary
section_names = []
section_texts = []
sections = soup.find_all('section', class_='ltx_section')
for section in sections:
    section_title = section.find('h2', class_='ltx_title ltx_title_section').text.strip()
    if section_title == 'References':
        continue
    section_names.append(section_title)
    section_text = ''
    # iterate all children recursively and cocatenate all the text in all the children and subchildren
    # skip if the child is a figure
    # if the tag is math, then use text in alttext
    valid_tags = ['p', 'math', 'cite','h3']
    valid_sections = ['div', 'section']
    for child in section.children:
        if child.name in valid_sections:
            for descendent in child.descendants:
                if descendent.name in valid_tags:
                    if descendent.name == 'math':
                        section_text += descendent['alttext']
                    if descendent.name == 'h3':
                        section_text += ' ' + descendent.text.strip() + '. '
                    else:
                        section_text += descendent.text.strip()
        

    section_texts.append(section_text)

In [47]:
# write the first section text to a file
with open('sample_section.txt', 'w', encoding='utf-8') as f:
    f.write(section_texts[1])

In [ ]:
# write everything into an dictionary
paper = {
    'title': title,
    'authors': authors,
    'abstract': abstract,
    'sections': dict(zip(section_names, section_texts))
}


In [7]:
url = "https://ar5iv.labs.arxiv.org/html/2305.04790"
from paper import Paper
paper = Paper(url, browser)
paper.parse_pdf()


TypeError: argument of type 'NoneType' is not iterable